<a href="https://colab.research.google.com/github/HaeunJeon/Jeju_Rec/blob/main/%ED%81%B4%EB%9F%AC%EC%8A%A4%ED%84%B0%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **파일불러오기**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
def bring():
  df = pd.read_excel("/content/drive/MyDrive/종합설계/final_review2.xlsx")
  df.to_csv
  df=df.loc[:,['place', 'review']]
  return df

# **군집화**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans
def clustering(content):
  # 군집화 할 그룹의 갯수 정의
  n_clusters = len(content)//50

    # CountVectrizer로 토큰화
  global vectorizer 
  vectorizer = CountVectorizer()
  global X 
  X = vectorizer.fit_transform(content)

    # l2 정규화
  X = normalize(X)

    # k-means 알고리즘 적용
  kmeans = KMeans(n_clusters=n_clusters).fit(X)

    # trained labels and cluster centers
  labels = kmeans.labels_
  centers = kmeans.cluster_centers_

    # labels에 merge
  place['labels'] = labels
  df = place
  return kmeans

# **빈도수**

In [ ]:
#빈도수 계산 위한 tfidf
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict

# ============================================
# -- Get TFIDF
# ============================================
def tfidf(content):
  # vectorizer = TfidfVectorizer()
  # sp_matrix = vectorizer.fit_transform(content)

  word2id = defaultdict(lambda : 0)
  for idx, feature in enumerate(vectorizer.get_feature_names()):
      word2id[feature] = idx

  for i, sent in enumerate(content):
      print('====== document[%d] ======' % i)
      print( [ (token, X[i, word2id[token]]) for token in sent.split() ] )
  return vectorizer

# **상위 feature 출력**

In [ ]:
def get_cluster_details(cluster_model, cluster_data, feature_names,
                       cluster_num, top_n_features=10):
    cluster_details = {}
    # 각 클러스터 레이블별 feature들의 center값들 내림차순으로 정렬 후의 인덱스를 반환
    center_feature_idx = cluster_model.cluster_centers_.argsort()[:,::-1]
    
    # 개별 클러스터 레이블별로 
    for cluster_num in range(cluster_num):
        # 개별 클러스터별 정보를 담을 empty dict할당
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster'] = cluster_num
        
        # 각 feature별 center값들 정렬한 인덱스 중 상위 20개만 추출
        top_ftr_idx = center_feature_idx[cluster_num, :top_n_features]
        top_ftr = [feature_names[idx] for idx in top_ftr_idx]
        # top_ftr_idx를 활용해서 상위 20개 feature들의 center값들 반환
        # 반환하게 되면 array이기 떄문에 리스트로바꾸기
        top_ftr_val = cluster_model.cluster_centers_[cluster_num, top_ftr_idx].tolist()
        
        # cluster_details 딕셔너리에다가 개별 군집 정보 넣어주기
        cluster_details[cluster_num]['top_features'] = top_ftr
        cluster_details[cluster_num]['top_featrues_value'] = top_ftr_val
        
    return cluster_details

def print_cluster_details(cluster_details):
    for cluster_num, cluster_detail in cluster_details.items():
        print()
        print(f"#####Cluster Num: {cluster_num}")
        print()
        print("상위 10개 feature단어들:\n", cluster_detail['top_features'])
        print()
        print('-'*20)

# **장소입력 및 실행**

In [ ]:
#장소입력
df = bring()
where=df.loc[:,['place']]
where.drop_duplicates(subset='place',inplace=True,ignore_index=True) #장소 중복제거 
where = where['place'].tolist()

In [ ]:
place = df.loc[df.place == where[0]]
content= place['review'].apply(lambda x : str(x))
kmeans = clustering(content)
vectorizer = tfidf(content)
feature_names = vectorizer.get_feature_names()
cluster_details = get_cluster_details(cluster_model=kmeans,
                                    cluster_data=content,
                                    feature_names=feature_names,
                                    cluster_num=len(content)//50,
                                    top_n_features=20)
print_cluster_details(cluster_details)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


====== document[0] ======
[('얼마', 0.20412414523193154), ('전', 0.0), ('가족들과', 0.20412414523193154), ('외식도', 0.20412414523193154), ('할', 0.0), ('겸', 0.0), ('도민들이', 0.20412414523193154), ('많이', 0.20412414523193154), ('방문한다는', 0.20412414523193154), ('곽지해수욕장', 0.4082482904638631), ('맛', 0.0), ('집에서', 0.20412414523193154), ('식사를', 0.20412414523193154), ('했다', 0.20412414523193154), ('맛과', 0.20412414523193154), ('가게', 0.20412414523193154), ('안으로', 0.20412414523193154), ('들어가니', 0.20412414523193154), ('은은한', 0.20412414523193154), ('조명과', 0.20412414523193154), ('원목', 0.20412414523193154), ('인테리어가', 0.20412414523193154), ('돋보이는', 0.20412414523193154), ('곽지해수욕장', 0.4082482904638631), ('맛', 0.0), ('집의', 0.20412414523193154), ('홀과', 0.20412414523193154)]
====== document[1] ======
[('헷', 0.0), ('코스', 0.19245008972987526), ('카페', 0.19245008972987526), ('청춘', 0.19245008972987526), ('부부', 0.19245008972987526), ('신창', 0.19245008972987526), ('풍차', 0.19245008972987526), ('해안도로', 0.19245008972987526), ('곽지해